In [1]:
%load_ext autoreload

In [36]:
import numpy as np
import os
import mcubes
import meshcat
import pydrake
from pydrake.geometry import SceneGraph
from pydrake.systems.framework import DiagramBuilder
from pydrake.common import FindResourceOrThrow
from pydrake.multibody.plant import MultibodyPlant, AddMultibodyPlantSceneGraph
from pydrake.multibody.parsing import Parser, LoadModelDirectives, ProcessModelDirectives
from pydrake.multibody.tree import RevoluteJoint
from pydrake.all import ConnectMeshcatVisualizer, InverseKinematics, RigidTransform

from meshcat import Visualizer
from functools import partial

import ipywidgets as widgets
from IPython.display import display

In [37]:
# Setup meshcat
from meshcat.servers.zmqserver import start_zmq_server_as_subprocess
proc, zmq_url, web_url = start_zmq_server_as_subprocess(server_args=[])
proc2, zmq_url2, web_url2 = start_zmq_server_as_subprocess(server_args=[])

In [38]:
#settings
q0 = [0.0, 0.0, 0.0]
q_low = [-1.7, -3.3, -1.7]
q_high = [1.7, 3.3, 1.7]

#marching cubes
q_low_mc = q_low.copy()
q_high_mc =  q_high.copy()
N = 20

In [40]:
vis = Visualizer(zmq_url=zmq_url)
vis.delete()
vis2 = Visualizer(zmq_url=zmq_url2)
vis2.delete()
#display(vis.jupyter_cell())

builder = DiagramBuilder()
plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=0.001)
parser = Parser(plant)
one_dof_asset = FindResourceOrThrow("drake/sandbox/assets/planar1dof.urdf")
two_dof_asset = FindResourceOrThrow("drake/sandbox/assets/planar2dof.urdf")
box_asset = FindResourceOrThrow("drake/sandbox/assets/box.urdf")

models =[]
models.append(parser.AddModelFromFile(two_dof_asset))
models.append(parser.AddModelFromFile(one_dof_asset))
#models.append(parser.AddModelFromFile(box_asset))

#directives_file = FindResourceOrThrow("drake/sandbox/planar_iiwa_simple_collision_welded_gripper.yaml") \
#    if simple_collision else FindResourceOrThrow("drake/sandbox/planar_iiwa_dense_collision_welded_gripper.yaml")
#directives = LoadModelDirectives(directives_file)
#models = ProcessModelDirectives(directives, plant, parser)


index = 0
for model in models:
    for joint_index in plant.GetJointIndices(model):
        joint = plant.get_mutable_joint(joint_index)
        if isinstance(joint, RevoluteJoint):
            joint.set_default_angle(q0[index])
            index += 1
for model in models:
    plant.WeldFrames(plant.world_frame(), plant.GetFrameByName("base", model), RigidTransform([0.0,0,0.0]))
plant.Finalize()

visualizer = ConnectMeshcatVisualizer(builder, scene_graph, zmq_url=zmq_url, delete_prefix_on_load=False)

diagram = builder.Build()
visualizer.load()
context = diagram.CreateDefaultContext()
plant_context = plant.GetMyContextFromRoot(context)
diagram.Publish(context)

joints = []
for model in models:
    jointindx = plant.GetJointIndices(model)
    for j in jointindx:
        joint = plant.get_mutable_joint(j)
        if isinstance(joint, RevoluteJoint):
            joints.append(joint)
    
def set_joint_ang(val, idx):
    joints[idx].set_angle(plant_context, val)
    
ik = InverseKinematics(plant, plant_context)
collision_constraint = ik.AddMinimumDistanceConstraint(0.001, 0.01)

def eval_cons(q0, q1, q2, c, tol):
        return 1-1*float(c.evaluator().CheckSatisfied([q0, q1, q2], tol))
    
col_func_handle = partial(eval_cons, c=collision_constraint, tol=0.01)

def showres(q):
    set_joint_ang(q[0],0)
    set_joint_ang(q[1],1)
    set_joint_ang(q[2],2)
    vis2["q"].set_object(
            meshcat.geometry.Sphere(0.2), meshcat.geometry.MeshLambertMaterial(color=0xFFB900))
    vis2["q"].set_transform(
            meshcat.transformations.translation_matrix(q))
    diagram.Publish(context)
    print("              ", end = "\r")
    print(col_func_handle(*q) , end = "\r")

sliders = []
sliders.append(widgets.FloatSlider(min=q_low[0], max=q_high[0], value=0, description='q0'))
sliders.append(widgets.FloatSlider(min=q_low[1], max=q_high[1], value=0, description='q1'))
sliders.append(widgets.FloatSlider(min=q_low[2], max=q_high[2], value=0, description='q2'))

q = q0.copy()
def handle_slider_change(change, idx):
    q[idx] = change['new']
    #print(q, end="\r")
    showres(q)
    
idx = 0
for slider in sliders:
    slider.observe(partial(handle_slider_change, idx = idx), names='value')
    idx+=1


You can open the visualizer by visiting the following URL:
http://127.0.0.1:7002/static/
You can open the visualizer by visiting the following URL:
http://127.0.0.1:7003/static/
Connecting to meshcat-server at zmq_url=tcp://127.0.0.1:6002...
You can open the visualizer by visiting the following URL:
http://127.0.0.1:7002/static/
Connected to meshcat-server.


In [41]:
#marching cubes
vertices, triangles = mcubes.marching_cubes_func(tuple(q_low), tuple(q_high), N, N, N, col_func_handle, 0.5)
vis2["collision_constraint"].set_object(
            meshcat.geometry.TriangularMeshGeometry(vertices, triangles),
            meshcat.geometry.MeshLambertMaterial(color=0xee8888, wireframe=False))
q = q0.copy()
showres(q)

In [42]:
for slider in sliders:
    display(slider)

display(vis.jupyter_cell())
display(vis2.jupyter_cell())

Widget Javascript not detected.  It may not be installed or enabled properly.


Widget Javascript not detected.  It may not be installed or enabled properly.


Widget Javascript not detected.  It may not be installed or enabled properly.


In [47]:
plant.CalcMassMatrix(context)

RuntimeError: Context was not created for drake::multibody::MultibodyPlant<double> system ::_::plant; it was created for system ::_